In [1]:
!pip install tensorflow==2.4.1   

In [2]:
!pip install tensorflow-gpu==2.4.1

In [3]:
!pip install tensorflow-hub

In [4]:
!pip install opencv-python matplotlib

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
cap = cv2.VideoCapture('C:\\Users\\User\\novak.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640) # 192 256
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1) #0.3
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#plt.imshow(frame)

In [28]:
 results['output_0'].numpy()[:,:,:51].reshape((6,17,3)) #(y,x,score * 17)

array([[[7.65526235e-01, 6.52963519e-01, 1.70355216e-01],
        [6.80348039e-01, 6.87467158e-01, 2.23887637e-01],
        [6.79663360e-01, 6.03413522e-01, 1.71073332e-01],
        [7.14429975e-01, 7.28903770e-01, 8.71129856e-02],
        [7.36652315e-01, 5.23035824e-01, 1.99948810e-02],
        [8.19265544e-01, 7.29393363e-01, 5.92180528e-03],
        [8.07242155e-01, 4.77194548e-01, 1.82278901e-02],
        [8.15371692e-01, 7.24579632e-01, 5.08164558e-05],
        [8.04157376e-01, 4.77026612e-01, 7.61439791e-04],
        [7.84912944e-01, 6.59221411e-01, 3.56216803e-02],
        [7.97469854e-01, 5.21741807e-01, 1.23837655e-02],
        [7.94801116e-01, 7.03563273e-01, 1.95411690e-06],
        [7.91571558e-01, 5.98784029e-01, 2.84397629e-07],
        [7.94218481e-01, 6.56791627e-01, 1.20455651e-02],
        [7.94379711e-01, 5.55806816e-01, 3.24555635e-02],
        [6.63472295e-01, 5.73038399e-01, 1.04145044e-02],
        [6.64950371e-01, 5.91768444e-01, 5.99489408e-03]],

       [[6.2

In [25]:
frame.shape

(480, 640, 3)

In [8]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [9]:
keypoints_with_scores.shape

NameError: name 'keypoints_with_scores' is not defined

In [4]:
#draw keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
keypoints_with_scores[0]

NameError: name 'keypoints_with_scores' is not defined

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)